In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
### Load Ann trained model, scalar pickle , one hot

model = load_model('model.h5')

## load the encoder and scalar

with open('ohehot_encoder_geo.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)
    
with open('scalar.pkl','rb') as file:
    scaler = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 6,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [4]:
input_df = pd.DataFrame([input_data])

In [5]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns = label_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [6]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [7]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,6,60000,2,1,1,50000


In [8]:
input_df = pd.concat([input_df.drop(columns = ['Geography']),geo_encoded_df],axis = 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,6,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
input_data = scaler.transform(input_df)

In [11]:
# Prediction
prediction_score = model.predict(input_data)
prediction_score

1/1 [==============================] - 0s 52ms/step


array([[0.02326835]], dtype=float32)

In [13]:
prediction_proba = prediction_score[0][0]
prediction_proba

0.023268353

In [14]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is not likely to churn.
